In [1]:
!pip install opendatasets --quiet

In [2]:
!nvidia-smi

Mon Nov 17 21:44:22 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   53C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: aravimenon
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/samithsachidanandan/human-face-emotions


100%|██████████| 701M/701M [00:00<00:00, 1.67GB/s]


In [115]:
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import numpy as np
import os
import PIL
import torch
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.amp import autocast
from torch.amp import GradScaler
import torchvision.transforms as transforms
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torchvision.models import resnet50, ResNet50_Weights, vgg16
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import tqdm
from sklearn.preprocessing import LabelEncoder
device = 'cuda' if torch.cuda.is_available() else 'cpu'
scaler = GradScaler()

In [116]:
import gc
gc.collect()

248

In [117]:
image_path = []
labels = []

data_base_path = "/content/human-face-emotions/Data" # Corrected path
for emotion in os.listdir(data_base_path):
    emotion_path = os.path.join(data_base_path, emotion)
    if os.path.isdir(emotion_path): # Ensure it's a directory
        for image_file in os.listdir(emotion_path):
            image_path.append(os.path.join(emotion_path, image_file))
            labels.append(emotion)

df = pd.DataFrame(zip(image_path, labels), columns=["image_path", "label"])

In [118]:
df.head()

,image_path,label
0,/content/human-face-emotions/Data/Happy/24123.png,Happy
1,/content/human-face-emotions/Data/Happy/382650...,Happy
2,/content/human-face-emotions/Data/Happy/754606...,Happy
3,/content/human-face-emotions/Data/Happy/203514...,Happy
4,/content/human-face-emotions/Data/Happy/382092...,Happy


In [119]:
train = df.sample(frac = 0.7)
test = df.drop(train.index)
val = test.sample(frac = 0.5)
test = test.drop(val.index)

In [120]:
encoder = LabelEncoder()
encoder.fit(df['label'])


train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(232),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
transforms = ResNet50_Weights.IMAGENET1K_V1.transforms()
#transforms = ViT_B_16_Weights.IMAGENET1K_SWAG_E2E_V1.transforms()

In [121]:
class CustomImageDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.labels = torch.tensor(encoder.fit_transform(dataframe['label']))
        self.transform = transform

    def __len__(self):
        return self.dataframe.shape[0]

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.labels[idx]
        image = PIL.Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [133]:
train_dataset = CustomImageDataset(train, transform = transforms)
test_dataset = CustomImageDataset(test,transform = val_transforms)
val_dataset = CustomImageDataset(val, transform = transforms)

In [134]:
BATCH_SIZE = 64
lr = 5e-4
EPOCHS = 100
num_classes = 5

In [135]:
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [97]:
import time

start = time.time()
for i, (inputs, labels) in enumerate(train_dataloader):
    if i >= 10:
        break
    inputs = inputs.to(device)
end = time.time()
print(f"10 batches took: {end - start:.2f} seconds")

10 batches took: 2.33 seconds


In [136]:
model = resnet50(weights='IMAGENET1K_V1')

In [137]:
model.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes)
)


In [138]:
for param in model.parameters():
  param.requires_grad = False

for param in model.fc.parameters():
  param.requires_grad = True


In [139]:
model = model.to(device)

In [140]:
class EarlyStopping:
  def __init__(self, patience = 1, min_delta = 0):
    self.patience = patience
    self.min_delta = min_delta
    self.counter = 0
    self.min_validation_loss = float("inf")

  def early_stop(self, validation_loss):
    if validation_loss < self.min_validation_loss:
      self.min_validation_loss = validation_loss
      self.counter = 0
    elif validation_loss > (self.min_validation_loss + self.min_delta):
      self.counter += 1
    if self.counter >= self.patience:
      return True
    return False


In [142]:
criterion = nn.CrossEntropyLoss(label_smoothing= 0.1)
optimizer = Adam(model.fc.parameters(), lr = 0.001, weight_decay= 1e-3)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 3)
early_stopping = EarlyStopping(patience = 4, min_delta = 0.001)

In [143]:
class Trainer:
   def __init__(self, model, criterion, optimizer, scheduler  = None, device = 'cuda', early_stopping = None):
      self.model = model
      self.criterion = criterion
      self.optimizer = optimizer
      self.device = device
      self.scheduler = scheduler
      self.early_stopping = early_stopping
      self.scaler = torch.amp.GradScaler()

      self.train_losses = []
      self.train_accs = []
      self.val_losses = []
      self.val_accs = []


   def train_model(self, dataloader):
        total_acc_train = 0
        total_loss_train = 0
        for inputs, labels in dataloader:
            self.optimizer.zero_grad()
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            with autocast(device_type = device):
              prediction = self.model(inputs)
              batch_loss = self.criterion(prediction, labels)

            total_loss_train += batch_loss.item()

            self.scaler.scale(batch_loss).backward()
            self.scaler.step(self.optimizer)
            self.scaler.update()

            acc = (torch.argmax(prediction, dim=1) == labels).sum().item()
            total_acc_train += acc

        avg_loss_train = total_loss_train / len(dataloader)
        avg_acc_train = (total_acc_train / len(dataloader.dataset)) * 100




        return avg_loss_train, avg_acc_train

   def validate(self, dataloader):
      total_loss_val = 0
      total_acc_val = 0
      with torch.no_grad():
          for inputs, labels in dataloader:
              inputs, labels = inputs.to(self.device), labels.to(self.device)

              with autocast(device_type = self.device):

                prediction = self.model(inputs)
                val_loss = criterion(prediction, labels)

              total_loss_val += val_loss.item()

              val_acc = (torch.argmax(prediction, dim=1) == labels).sum().item()
              total_acc_val += val_acc

      avg_loss_val = total_loss_val / len(dataloader)
      avg_acc_val = (total_acc_val / len(dataloader.dataset)) * 100

      return avg_loss_val, avg_acc_val

   def fit(self, train_dataloader, val_dataloader, epochs):
        for epoch in range(epochs):
            train_loss, train_acc = self.train_model(train_dataloader)
            val_loss, val_acc = self.validate(val_dataloader)
            self.train_losses.append(train_loss)
            self.train_accs.append(train_acc)
            self.val_losses.append(val_loss)
            self.val_accs.append(val_acc)

            self.scheduler.step(val_loss)

            current_lr = self.optimizer.param_groups[0]['lr']

            print(f'''Epoch: {epoch + 1}, Train Loss: {train_loss:.2f}, Val Loss: {val_loss:.2f}, Train Acc: {train_acc:.2f}, Val Acc: {val_acc:.2f}, Current LR: {current_lr}''')
            if self.early_stopping.early_stop(val_loss):
              print(f"Early stopping initiated at epoch {epoch + 1}")
              break

   def plot_artifacts(self):
        fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (15,5))
        axs[0].plot(self.train_losses, label = "training_loss")
        axs[0].plot(self.val_losses, label = "validation loss")
        axs[0].set_title("Training and validation loss over epochs")
        axs[0].set_xlabel("Epochs")
        axs[0].set_ylabel("Loss")
        axs[0].set_ylim([0,2])
        axs[0].legend()


        axs[1].plot(self.train_accs, label = "training accuracy")
        axs[1].plot(self.val_accs, label = "validation accuracy")
        axs[1].set_title("Training and validation accuracy over epochs")
        axs[1].set_xlabel("Epochs")
        axs[1].set_ylabel("Accuracy")
        axs[1].set_ylim([0,100])
        axs[1].legend()

        plt.show()






In [144]:
trainer = Trainer(model, criterion, optimizer, scheduler, device, early_stopping)
trainer.fit(train_dataloader, val_dataloader, epochs = EPOCHS)

Epoch: 1, Train Loss: 1.31, Val Loss: 1.27, Train Acc: 50.55, Val Acc: 52.70, Current LR: 0.001
Epoch: 2, Train Loss: 1.20, Val Loss: 1.22, Train Acc: 56.88, Val Acc: 55.62, Current LR: 0.001
Epoch: 3, Train Loss: 1.14, Val Loss: 1.19, Train Acc: 60.46, Val Acc: 57.68, Current LR: 0.001
Epoch: 4, Train Loss: 1.09, Val Loss: 1.17, Train Acc: 63.54, Val Acc: 58.69, Current LR: 0.001
Epoch: 5, Train Loss: 1.05, Val Loss: 1.16, Train Acc: 66.23, Val Acc: 60.49, Current LR: 0.001
Epoch: 6, Train Loss: 1.01, Val Loss: 1.14, Train Acc: 68.76, Val Acc: 61.96, Current LR: 0.001
Epoch: 7, Train Loss: 0.98, Val Loss: 1.13, Train Acc: 70.32, Val Acc: 61.74, Current LR: 0.001
Epoch: 8, Train Loss: 0.94, Val Loss: 1.11, Train Acc: 72.68, Val Acc: 64.62, Current LR: 0.001
Epoch: 9, Train Loss: 0.92, Val Loss: 1.09, Train Acc: 74.26, Val Acc: 65.18, Current LR: 0.001
Epoch: 10, Train Loss: 0.89, Val Loss: 1.09, Train Acc: 75.97, Val Acc: 65.37, Current LR: 0.001
Epoch: 11, Train Loss: 0.87, Val Loss: 

In [146]:
for param in model.layer4.parameters():
  param.requires_grad = True


In [150]:
optimizer2 = Adam(list(model.fc.parameters()) +
                  list(model.layer4.parameters()), lr = 3e-4, weight_decay = 2e-4)

early_stopping2 = EarlyStopping(patience = 4, min_delta = 0.001)
scheduler2 = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 3)

In [151]:
trainer2 = Trainer(model, criterion, optimizer2, scheduler2, device, early_stopping2)
trainer2.fit(train_dataloader, val_dataloader, epochs = EPOCHS)

Epoch: 1, Train Loss: 0.89, Val Loss: 0.86, Train Acc: 76.78, Val Acc: 78.76, Current LR: 0.0003
Epoch: 2, Train Loss: 0.70, Val Loss: 0.80, Train Acc: 87.39, Val Acc: 82.54, Current LR: 0.0003
Epoch: 3, Train Loss: 0.59, Val Loss: 0.75, Train Acc: 93.23, Val Acc: 85.74, Current LR: 0.0003
Epoch: 4, Train Loss: 0.54, Val Loss: 0.73, Train Acc: 95.45, Val Acc: 86.90, Current LR: 0.0003
Epoch: 5, Train Loss: 0.52, Val Loss: 0.71, Train Acc: 96.20, Val Acc: 87.46, Current LR: 0.0003
Epoch: 6, Train Loss: 0.50, Val Loss: 0.72, Train Acc: 96.69, Val Acc: 87.50, Current LR: 0.0003
Epoch: 7, Train Loss: 0.49, Val Loss: 0.70, Train Acc: 96.89, Val Acc: 87.76, Current LR: 0.0003
Epoch: 8, Train Loss: 0.49, Val Loss: 0.70, Train Acc: 97.12, Val Acc: 88.04, Current LR: 0.0003
Epoch: 9, Train Loss: 0.48, Val Loss: 0.70, Train Acc: 97.40, Val Acc: 87.79, Current LR: 0.0003
Epoch: 10, Train Loss: 0.47, Val Loss: 0.69, Train Acc: 97.44, Val Acc: 87.29, Current LR: 0.0003
Epoch: 11, Train Loss: 0.47, 

In [152]:
for param in model.layer3.parameters():
  param.requires_grad = True

In [155]:
optimizer3 = Adam(list(model.fc.parameters()) +
                  list(model.layer3.parameters()) +
                  list(model.layer4.parameters()), lr = 2e-6
                  , weight_decay = 2e-5)

early_stopping3 = EarlyStopping(patience = 4, min_delta = 0.001)
scheduler3 = ReduceLROnPlateau(optimizer, mode = 'min', factor = 0.5, patience = 3)

In [156]:
trainer3 = Trainer(model, criterion, optimizer3, scheduler3, device, early_stopping3)
trainer3.fit(train_dataloader, val_dataloader, epochs = EPOCHS)

Epoch: 1, Train Loss: 0.40, Val Loss: 0.64, Train Acc: 99.81, Val Acc: 89.98, Current LR: 2e-06
Epoch: 2, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.81, Val Acc: 90.22, Current LR: 2e-06
Epoch: 3, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.82, Val Acc: 89.93, Current LR: 2e-06
Epoch: 4, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.82, Val Acc: 90.15, Current LR: 2e-06
Epoch: 5, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.81, Val Acc: 90.12, Current LR: 2e-06
Epoch: 6, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.83, Val Acc: 90.11, Current LR: 2e-06
Epoch: 7, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.84, Val Acc: 89.98, Current LR: 2e-06
Epoch: 8, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.83, Val Acc: 90.11, Current LR: 2e-06
Epoch: 9, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.83, Val Acc: 90.04, Current LR: 2e-06
Epoch: 10, Train Loss: 0.40, Val Loss: 0.63, Train Acc: 99.83, Val Acc: 89.85, Current LR: 2e-06
Epoch: 11, Train Loss: 0.40, Val Loss: 

In [157]:
save_path = "/content/drive/MyDrive/human_emotions/models/emotion_model_final.pth"
torch.save(model.state_dict(), save_path)
print(f"Model saved to: {save_path}")

Model saved to: /content/drive/MyDrive/human_emotions/models/emotion_model_final.pth


In [24]:
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable: {trainable:,} / Total: {total:,}")

Trainable: 112,232,453 / Total: 117,507,909


In [ ]:
torch.save(model.state_dict(), 'resnet_model.pth')
